Install Data.World

$ pip install git+git://github.com/datadotworld/data.world-py.git

Then, go to https://data.world/settings/advanced and get your API token
    
Then simply run the following in Terminal and enter one time Token

$ dw configure

In [1]:
import datadotworld as dw
import pandas as pd
import numpy as np
import pprint as pp
import os

In [2]:
help(dw)

Help on package datadotworld:

NAME
    datadotworld - A python library for working with data.world datasets

PACKAGE CONTENTS
    cli
    client (package)
    config
    datadotworld
    files
    models (package)
    util

FUNCTIONS
    api_client(profile='default', **kwargs)
        Return API client for access to data.world's REST API
        
        :param profile: Configuration profile (account) to use.
            (Default value = 'default')
        :type profile: str, optional
        :returns: REST API client object
        :rtype: RestApiClient
        
        Examples
        --------
        >>> import datadotworld as dw
        >>> client = dw.api_client()
        >>> client.get_dataset(
        ...     'jonloyens/an-intro-to-dataworld-dataset').get('title')
        'An Intro to data.world Dataset'
    
    load_dataset(dataset_key, force_update=False, auto_update=False, profile='default', **kwargs)
        Load a dataset from the local filesystem, downloading it from da

In [3]:
#Load Meta Data from data.world
dataset = dw.load_dataset('sketchcity/city-of-houston-email-metadata-january-march-2017')
pp.pprint(dataset.describe())

{'description': 'Metadata from houstontx.gov email addresses\n'
                '\n'
                'Data includes:\n'
                '- Sender\n'
                '- To\n'
                '- CC\n'
                '- BCC\n'
                '- Sent date/time\n'
                '- Received date/time\n'
                '\n'
                'Assembled by Matt Chapman (hubblefisher@gmail.com) through '
                'FOIA request.',
 'homepage': 'https://data.world/sketchcity/city-of-houston-email-metadata-january-march-2017',
 'license': 'Public Domain',
 'name': 'sketchcity_city-of-houston-email-metadata-january-march-2017',
 'resources': [{'bytes': 1370801133,
                'format': 'csv',
                'mediatype': 'text/csv',
                'name': 'original/coh_email_metadata_1Q17.csv',
                'path': 'original/coh_email_metadata_1Q17.csv'}],
 'title': 'City of Houston Email Metadata: 1/1/17 - 2/23/17'}


In [4]:
#Import data
from io import BytesIO
import csv
df = pd.read_csv(BytesIO(dataset.raw_data['original/coh_email_metadata_1Q17.csv']))

In [5]:
#View Data size, Null value counts and sample data
print(df.shape)
print(df.isnull().sum())
df.head()

(6090750, 6)
Sender           30
To            49161
CC          5187844
BCC         5906625
Sent            173
Received        173
dtype: int64


,Sender,To,CC,BCC,Sent,Received
0,Houston Parks and Recreation Department <Houst...,"Lathan, Debra - PRD <Debra.Lathan@houstontx.gov>",NaN,NaN,2017-02-14 06:55:54Z,2017-02-14 06:55:52Z
1,Houston Parks and Recreation Department <Houst...,"Lathan, Debra - PRD <Debra.Lathan@houstontx.gov>",NaN,NaN,2017-02-14 06:55:54Z,2017-02-14 06:55:52Z
2,"DIEP, BA <Ba.Diep@tsa.dhs.gov>",HOUcoordinationcenter <HOUcoordinationcenter@t...,NaN,NaN,2017-02-14 06:57:22Z,2017-02-14 06:57:31Z
3,has.nicealerts@houstontx.gov <has.nicealerts@h...,"Ulrich, David - HAS <David.Ulrich@houstontx.gov>",NaN,NaN,2017-02-14 06:57:54Z,2017-02-14 06:57:54Z
4,has.movi@has.net <has.movi@has.net>,"Lueders, Ulf - HAS <Ulf.Lueders@houstontx.gov>",NaN,NaN,2017-02-14 06:58:01Z,2017-02-14 06:58:01Z


In [6]:
#Delete Null rows
df.dropna(subset=['Sender','To','Sent','Received'], how='any', inplace=True)
df = df.reset_index(drop=True)
print(df.shape)
print(df.isnull().sum())

(6041532, 6)
Sender            0
To                0
CC          5143103
BCC         5904214
Sent              0
Received          0
dtype: int64


In [7]:
#Dropping rows with incorrect times(some rows contain email ids in time columns)
df = df.drop(df.index[[x for x in range(df.shape[0]) if (df['Sent'][x][:5]!='2017-' or df['Received'][x][:5]!='2017-')]]).reset_index(drop=True)
df.shape

(6041353, 6)

In [8]:
import datetime
df['Sent'] = df['Sent'].apply(lambda x: datetime.datetime.strptime(x[:-1], '%Y-%m-%d %H:%M:%S'))
df['Received'] = df['Received'].apply(lambda x: datetime.datetime.strptime(x[:-1], '%Y-%m-%d %H:%M:%S'))

In [9]:
df['Sender_Counts'] = df.groupby(['Sender'])['Sender'].transform('count')
df['Receiver_Counts'] = df.groupby(['To'])['To'].transform('count')
df['CC_Counts'] = df.groupby(['CC'])['CC'].transform('count')
df['BCC_Counts'] = df.groupby(['BCC'])['BCC'].transform('count')
df['Sent_Time_Counts'] = df.groupby(['Sent'])['Sent'].transform('count')
df['Receiver_Time_Counts'] = df.groupby(['Received'])['Received'].transform('count')

In [10]:
df

,Sender,To,CC,BCC,Sent,Received,Sender_Counts,Receiver_Counts,CC_Counts,BCC_Counts,Sent_Time_Counts,Receiver_Time_Counts
0,Houston Parks and Recreation Department <Houst...,"Lathan, Debra - PRD <Debra.Lathan@houstontx.gov>",NaN,NaN,2017-02-14 06:55:54,2017-02-14 06:55:52,3741,2617,NaN,NaN,1970-01-01 00:00:00.000000003,1970-01-01 00:00:00.000000007
1,Houston Parks and Recreation Department <Houst...,"Lathan, Debra - PRD <Debra.Lathan@houstontx.gov>",NaN,NaN,2017-02-14 06:55:54,2017-02-14 06:55:52,3741,2617,NaN,NaN,1970-01-01 00:00:00.000000003,1970-01-01 00:00:00.000000007
2,"DIEP, BA <Ba.Diep@tsa.dhs.gov>",HOUcoordinationcenter <HOUcoordinationcenter@t...,NaN,NaN,2017-02-14 06:57:22,2017-02-14 06:57:31,62,8,NaN,NaN,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000001
3,has.nicealerts@houstontx.gov <has.nicealerts@h...,"Ulrich, David - HAS <David.Ulrich@houstontx.gov>",NaN,NaN,2017-02-14 06:57:54,2017-02-14 06:57:54,996,1461,NaN,NaN,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000001
4,has.movi@has.net <has.movi@has.net>,"Lueders, Ulf - HAS <Ulf.Lueders@houstontx.gov>",NaN,NaN,2017-02-14 06:58:01,2017-02-14 06:58:01,29865,30242,NaN,NaN,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000001
5,"Roberto, Francis - HAS <Francis.Roberto@housto...","Segura, Oscar - HAS <Oscar.Segura@houstontx.gov>",NaN,NaN,2017-02-14 06:58:11,2017-02-14 06:58:16,8,163,NaN,NaN,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000001
6,has.movi@has.net <has.movi@has.net>,"Lueders, Ulf - HAS <Ulf.Lueders@houstontx.gov>",NaN,NaN,2017-02-14 06:58:41,2017-02-14 06:58:41,29865,30242,NaN,NaN,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000001
7,Baker & Taylor Customer Service <btinfo@btol.com>,"Reyes, Esther - HPL <Esther.Reyes@houstontx.gov>",NaN,NaN,2017-02-14 07:00:07,2017-02-14 07:00:09,164,938,NaN,NaN,1970-01-01 00:00:00.000000039,1970-01-01 00:00:00.000000020
8,Baker & Taylor Customer Service <btinfo@btol.com>,"Bote, Meryl - HPL <Meryl.Bote@houstontx.gov>",NaN,NaN,2017-02-14 07:00:07,2017-02-14 07:00:10,164,522,NaN,NaN,1970-01-01 00:00:00.000000039,1970-01-01 00:00:00.000000012
9,Baker & Taylor Customer Service <btinfo@btol.com>,"Garza, Marla - HPL <Marla.Garza@houstontx.gov>",NaN,NaN,2017-02-14 07:00:07,2017-02-14 07:00:10,164,180,NaN,NaN,1970-01-01 00:00:00.000000039,1970-01-01 00:00:00.000000012


In [11]:
dfsummary = df.describe()
dfsummary

,Sender_Counts,Receiver_Counts,CC_Counts,BCC_Counts
count,6.041353e+06,6.041353e+06,898257.000000,137147.000000
mean,2.161915e+04,4.589706e+03,395.699199,840.478137
std,4.839114e+04,1.540025e+04,1997.768859,1507.242409
min,1.000000e+00,1.000000e+00,1.000000,1.000000
25%,2.240000e+02,1.090000e+02,3.000000,27.000000
50%,8.360000e+02,5.730000e+02,16.000000,161.000000
75%,9.703000e+03,1.371000e+03,117.000000,892.000000
max,1.921520e+05,9.949500e+04,15259.000000,6196.000000


In [12]:
import motplotlib as plt
%matplotlib inline
df.hist(bins=100,figsize=(20,15))
plt.show()

ModuleNotFoundError: No module named 'motplotlib'

In [ ]:
#Grouping senders based on their email sending volume
#If sender sends emails more than 1 std above mean - high sender, 1std below mean - low sender
df['Sender_Group']=''
for i in range(df.shape[0]):
    if df.iloc[i]['Sender_Counts'] != 'Null':
        if df.iloc[i]['Sender_Counts'] >= dfsummary['Sender_Counts'][1]+dfsummary['Sender_Counts'][2]:
            df.iloc[i]['Sender_Group'] = 'High_Sender'
        elif:
            if df.iloc[i]['Sender_Counts'] <= dfsummary['Sender_Counts'][1]-dfsummary['Sender_Counts'][2]:
                df.iloc[i]['Sender_Group'] = 'Low_Sender'
        else:
            df.iloc[i]['Sender_Group'] = 'Normal_Sender'
    else:
        df.iloc[i]['Sender_Group'] = 'Null'

df.columns.values[6] = 'Sender_Group'

In [ ]:
df

In [ ]:
df1 = df[['To','CC','BCC']].to_dict()
df1

In [ ]:
#Splitting the multiple email IDs in TO, CC, BCC columns and saving in dictionaries
for i in range(len(df1['To'])):
    j = str(df1['To'][i].split(';'))
    df1['To'][i] = {}
    for k in range(len(j)):
        df1['To'][i][k] = j[k]     
        
for i in range(len(df1['CC'])):
    j = str(df1['CC'][i]).split(';')
    df1['CC'][i] = {}
    for k in range(len(j)):
        df1['CC'][i][k] = j[k]    
        
for i in range(len(df1['BCC'])):
    j = str(df1['BCC'][i]).split(';')
    df1['BCC'][i] = {}
    for k in range(len(j)):
        df1['BCC'][i][k] = j[k]

In [ ]:
df1